In [ ]:
import pandas as pd


In [ ]:
data=pd.read_csv("https://raw.githubusercontent.com/VK-VCS/Models/main/winequality_red.csv")
data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
data['quality'].unique() #6 classes

array([5, 6, 7, 4, 8, 3])

In [ ]:
data.describe() #all are numeric, no missing data found

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [ ]:
X = data.drop(columns=['quality'])
y = data['quality']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 50 % data for base models train and test, rest 50% for validation of them
X_main,X_Validation,y_main,y_validation = train_test_split(X,y,test_size=0.5, random_state=200)

#train,test split for base models
x_train,x_test,y_train,y_test = train_test_split(X_main,y_main,test_size=0.2, random_state=200)

In [ ]:
#Model 1 - Decision Tree
from sklearn.tree import DecisionTreeClassifier

DT_clf = DecisionTreeClassifier(random_state=200)
DT_clf.fit(x_train,y_train)
DT_clf.score(x_test,y_test)

0.5375

In [ ]:
#Model 2 - KNN
from sklearn.neighbors import KNeighborsClassifier

KNN_clf = KNeighborsClassifier()
KNN_clf.fit(x_train,y_train)
KNN_clf.score(x_test,y_test)

0.43125

In [14]:
#Model 3 - Naive Bayes
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()
NB_clf.fit(x_train,y_train)
NB_clf.score(x_test,y_test)

0.43125

In [15]:
#Model 4 - Support Vector Machine

from sklearn.svm import SVC

SV_clf = SVC()
SV_clf.fit(x_train,y_train)
SV_clf.score(x_test,y_test)

0.48125

In [23]:
# predict ouputs of Base Models with validation datset
pred1 = DT_clf.predict(X_Validation)
pred2 = KNN_clf.predict(X_Validation)
pred3 = NB_clf.predict(X_Validation)
pred4 = SV_clf.predict(X_Validation)

#stack all the predictions column wise which will be feeded to Meta Model
import numpy as np
base_pred = np.column_stack((pred1,pred2,pred3,pred4))

#base_pred - train data for meta model

# predict ouputs of Base Models with test datset
pred_test1 = DT_clf.predict(x_test)
pred_test2 = KNN_clf.predict(x_test)
pred_test3 = NB_clf.predict(x_test)
pred_test4 = SV_clf.predict(x_test)

#stack all the predictions column wise which will be feeded to Meta Model
import numpy as np
base_pred_test = np.column_stack((pred_test1,pred_test2,pred_test3,pred_test4))

#base_pred_test - test data for meta model

In [24]:
#Meta Model - Inputs - X = base_pred, y = y_validation
from sklearn.ensemble import RandomForestClassifier

meta_clf = RandomForestClassifier(max_depth=2, random_state=200)
meta_clf.fit(base_pred, y_validation)
meta_clf.score(base_pred_test,y_test)

0.56875